In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing string and regular expression library

In [ ]:
import string
import re

# Read data and put it to dataframe

In [ ]:
df = pd.read_csv('../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')
tweets = df.tweet.tolist()
df.head(200)

# Remove duplicate tweets

Removing duplicates using set operation in pyhton.

In [ ]:
print(f"Number of tweets before removing dupplicates: {len(tweets)}")
tweets = list(set(tweets)) 
print(f"Number of tweets after removing dupplicates: {len(tweets)}")
tweets[0:10]



# Preprocessing of data

For the preprocessing, i have used regular expression to remove puntuations and url tags. I have also changed tweets to lowercase. Taken about just last tweets to speeden up computations.

In [ ]:
tweets = tweets[4000:]

for i in range(len(tweets)):
    tweets[i] = tweets[i].lower()
    tweets[i] = re.sub(r'http\S+', '', tweets[i])
    tweets[i] = re.sub(r'[^\w\s]', '', tweets[i]) 
tweets[0:20]

# Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)
total_no_words = len(tokenizer.word_index) + 1
total_no_words

# Input Sequence

Arranging the words to be represented by numbers instead of string representations.

In [ ]:
input_tweet_sequences = []
for tweet in tweets:
    tokenlist = tokenizer.texts_to_sequences([tweet])[0]
    for i in range(1, len(tokenlist)):
        ngram_sequence = tokenlist[:i+1]
        input_tweet_sequences.append(ngram_sequence)
        
print(input_tweet_sequences)

# Add padding

Providing all the sequence the same length.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_input_sequence_len = max([len(x) for x in input_tweet_sequences])
input_tweet_sequences = np.array(pad_sequences(input_tweet_sequences, maxlen=max_input_sequence_len, padding='pre'))

print(max_input_sequence_len)
print(input_tweet_sequences[0:10])

# Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model = Sequential()
model.add(Embedding(total_no_words, 160, input_length=max_input_sequence_len-1))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_no_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(total_no_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
import tensorflow.keras.utils as ku 

model_predictors, output_label = input_tweet_sequences[:,:-1],input_tweet_sequences[:,-1]
output_label = ku.to_categorical(output_label, num_classes=total_no_words)

history = model.fit(model_predictors, output_label, epochs=100, verbose=1, callbacks=[es])

In [ ]:
def Song_Generate(text, next_words):
    for _ in range(next_words):
        tokenlist = tokenizer.texts_to_sequences([text])[0]
        tokenlist = pad_sequences([tokenlist],
                     maxlen=max_input_sequence_len-1,padding='pre')
        predicted = model.predict_classes(tokenlist, verbose=0)
        output= ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output = word
                break
        text += " " + output
    print(text)

In [ ]:
next_words = 30 
text = "Democrats and Obama have developed"
Song_Generate(text, next_words)

# Visualizing history of epochs

In [ ]:
import matplotlib.pyplot as plt
def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()
plot_graph(history, 'accuracy')

It's one of my first notebooks. Feel free to comment and leave some suggestion how could i improve it :D